In [27]:
import pandas as pd
import typing as t
import os

DATASETS = {
    "S-FMNIST", "S-CIFAR10", "S-CIFAR100", "S-CORe50", "SE-CIFAR100", "SE-CORe50"
}


In [28]:
# for all files in the directory

os.chdir("experiment_logs")
for filename in os.listdir("."):
    num, host, repo_hash, label, dataset, model, strategy = filename.split("_")

    if strategy == "cumulative":
        os.rename(filename, f"{num}_{host}_{repo_hash}_{label}_{dataset}_{model}_nonContinual")
os.chdir("..")

In [29]:
def load_clean_table():
    exp_data = pd.read_csv("results/all_experiments.csv")

    def repo_version_filter(df: pd.DataFrame, versions: t.Dict[str, t.Set[str]]) -> pd.DataFrame:
        """Filter a dataframe by repo version"""

        def _is_valid(row):
            if row["dataset"] in versions:
                return row["repo_hash"] in versions[row["dataset"]]
            return False
        return df[df.apply(_is_valid, axis=1)]

    def did_complete(row) -> bool:
        """Check if an experiment has completed"""
        return row["completed_tasks"] == row["n_experiences"]

    def remove_unused_parameters(row):
        """Remove unused hyperparameter from rows"""
        if not row["use_packnet"]:
            row["prune_proportion"] = ""
        elif row["prune_proportion"][0] == "[":
            row["prune_proportion"] = "ep"
        if not row["use_experience_replay"]:
            row["replay_buffer"] = ""
        return row

    exp_data = repo_version_filter(
            exp_data,
            {
                "S-FMNIST": {"1555acb6", "0a229afa", "4e7023cd", "4e7023cdD", "0a229afaD", "54dcf601"},
                "S-CIFAR10": {"1555acb6", "0a229afa", "4e7023cd", "4e7023cdD", "0a229afaD"},
                "S-CIFAR100": {"1555acb6", "0a229afa", "4e7023cd", "4e7023cdD", "0a229afaD"},
                "S-CORe50": {"1555acb6", "4e7023cd", "4e7023cdD"},
                "SE-CIFAR100": {"1555acb6", "0a229afa", "4e7023cd", "4e7023cdD", "0a229afaD"},
                "SE-CORe50": {"1555acb6", "4e7023cd", "4e7023cdD"},
            },
            
    )
    # remove certain experiment categories
    exp_data = exp_data[~exp_data["experiment_category"].isin(["TEST"])]
    exp_data = exp_data[~exp_data["strategy"].isin(["LwF", "SI"])]
    # Filter out incomplete experiments
    exp_data = exp_data[exp_data.apply(did_complete, axis=1)]
    # Simplify hyper-parameters
    exp_data = exp_data.apply(remove_unused_parameters, axis=1)

    # exp_data = hyper_parameters(exp_data)
    # print(exp_data.columns)
    # pd.set_option('display.max_rows', None)

    group_by_keys = [
        "strategy",
        "architecture",
        "replay_buffer",
        "prune_proportion",
        "architecture",
        "dataset",]

    """
    This is 
    """
    grouped = exp_data.groupby(group_by_keys).sample(10, replace=False, random_state=0).groupby(group_by_keys)
    table_components = {
        "final_accuracy_mean": grouped["final_accuracy"].mean(),
        "final_accuracy_std": grouped["final_accuracy"].std(),
        "count": grouped["experiment_code"].count()
    }

    table = pd.DataFrame(
        table_components
    )

    table = table.pivot_table(
            values=["final_accuracy_mean", "final_accuracy_std", "count"], 
            columns="dataset", 
            index=['strategy', 'replay_buffer', 'prune_proportion', 'architecture'])
    # table.reindex()
    return table
clean_table = load_clean_table()

In [30]:
# Add data from different repositories

from os import PathLike


def add_csv(df: pd.DataFrame, index: t.Tuple, csv_file: PathLike, ):
    data = pd.read_csv(csv_file, index_col=[0], comment="#")

    # Add rows
    for dataset in DATASETS:
        if dataset not in data.columns:
            continue
        df.loc[index, ("final_accuracy_mean",dataset)] = data.mean()[dataset]
        df.loc[index, ("final_accuracy_std",dataset)] = data.std()[dataset]
        df.loc[index, ("count", dataset)] = data.count()[dataset]

    return df
clean_table = add_csv(clean_table, ("SnB", "1000", "", "FF"), "results/SnB.csv")
clean_table = add_csv(clean_table, ("BIR", "", "", "VAE"), "results/BIR.csv")
clean_table = add_csv(clean_table, ("GR", "", "", "VAE"), "results/GR.csv")


In [31]:
clean_table = clean_table.reindex(["nonContinual", "taskOracle", "finetuning", "replay", "SnB", "GR", "BIR", "taskInference"], level="strategy")

In [32]:
DISPLAY_STRATEGY_NAMES = {
    "nonContinual": "Non-Continual",
    "taskOracle": "Task Oracle PackNet \cite{Mallya_Lazebnik_2018}",
    "replay": "Experience Replay",
    "finetuning": "Finetuning",
    "SnB": "Split and Bridge \cite{Kim_Choi_2021}",
    "GR": "Generative Replay \cite{vandevenBraininspiredReplayContinual2020}",
    "BIR": "Brain Inspired Replay \cite{vandevenBraininspiredReplayContinual2020}",
    "taskInference": "CI-PackNet (ours)"
}

In [33]:
clean_table

count  \
dataset                                                   S-CIFAR10   
strategy      replay_buffer prune_proportion architecture             
nonContinual                                 AE                10.0   
                                             VAE               10.0   
taskOracle                  0.5              AE                10.0   
finetuning                                   AE                10.0   
                                             VAE               10.0   
replay        100.0                          AE                10.0   
              1000.0                         AE                10.0   
              10000.0                        AE                10.0   
SnB           1000                           FF                10.0   
GR                                           VAE               10.0   
BIR                                          VAE               10.0   
taskInference               0.2              AE                10.0   
                                             VAE               10.0   
                            0.4              AE                10.0   
                                             VAE               10.0   
                            0.5              AE                10.0   
                                             VAE               10.0   
                            0.6              AE                10.0   
                                             VAE               10.0   
                            0.8              AE                10.0   
                                             VAE               10.0   
                            ep               AE                10.0   
                                             VAE               10.0   

                                                                               \
dataset                                                   S-CIFAR100 S-CORe50   
strategy      replay_buffer prune_proportion architecture                       
nonContinual                                 AE                 10.0     10.0   
                                             VAE                10.0     10.0   
taskOracle                  0.5              AE                 10.0     10.0   
finetuning                                   AE                 10.0     10.0   
                                             VAE                10.0     10.0   
replay        100.0                          AE                 10.0     10.0   
              1000.0                         AE                 10.0     10.0   
              10000.0                        AE                 10.0     10.0   
SnB           1000                           FF                 10.0     10.0   
GR                                           VAE                10.0     10.0   
BIR                                          VAE                10.0     10.0   
taskInference               0.2              AE                 10.0     10.0   
                                             VAE                10.0     10.0   
                            0.4              AE                 10.0     10.0   
                                             VAE                10.0     10.0   
                            0.5              AE                 10.0     10.0   
                                             VAE                10.0     10.0   
                            0.6              AE                 10.0     10.0   
                                             VAE                10.0     10.0   
                            0.8              AE                 10.0     10.0   
                                             VAE                10.0     10.0   
                            ep               AE                 10.0     10.0   
                                             VAE                10.0     10.0   

                                                                    \
dataset                               

In [34]:
from pandas.io.formats.style import Styler
from IPython.display import HTML
import numpy as np

# print(clean_table.index)


def create_latex_table(
        df: pd.DataFrame,
        caption: str = "",
        label: str = "",
        bolding_ignores_rows = []) -> str:
    """Convert a dataframe to latex"""

    # Make the index more readable
    df = df.reset_index()
    def _format_hp(row):
        hp = ""
        prune_proportion = row["prune_proportion"][0]
        if prune_proportion == "ep":
            hp = "Equal Prune"
        elif prune_proportion == "Best":
            hp = "Best"
        elif prune_proportion != "":
            hp = f"$\lambda$={float(prune_proportion)*100:.0f}\%"

        replay_buffer = row["replay_buffer"][0]
        if replay_buffer != "":
            hp += f"$n$={int(replay_buffer)}"
        return hp
    # The new index will consist of the strategy, hyper-parameters, and the architecture
    df["Hyper-Parameters"] = df.apply(_format_hp, axis=1)
    df["Strategy"]         = df["strategy"].apply(lambda x: DISPLAY_STRATEGY_NAMES[x])
    df["Architecture"]     = df["architecture"]
    df = df.drop(columns=["strategy", "replay_buffer", "prune_proportion", "architecture"])
    df.set_index(["Strategy", "Hyper-Parameters", "Architecture"], inplace=True)
    
    
    # Format the values into a format like '81.6±8.6'
    def _format_values(row):
        pretty_row = {}
        for dataset, value in row.groupby("dataset"):
            final_accuracy_mean = value["final_accuracy_mean"][0] * 100
            final_accuracy_std = value["final_accuracy_std"][0] * 100
            if not np.isnan(final_accuracy_mean) and not np.isnan(final_accuracy_std):
                pretty_row[dataset] = f"{final_accuracy_mean:.1f}$\pm${final_accuracy_std:.2f}"
            else:
                pretty_row[dataset] = "-"
            
        return pd.Series(pretty_row)
    
    # Save the indices of the best cells for each dataset
    relevant_rows = np.setdiff1d(list(range(df.shape[0])), bolding_ignores_rows)
    best_rows = df["final_accuracy_mean"].iloc[relevant_rows].idxmax()


    df = df.apply(_format_values, axis=1)
    style: Styler = df.style

    # Bold the best cells
    def _bold_best(row):
        return ["font-weight: bold" if row.name == best_row else "" for best_row in best_rows]
    style = style.apply(_bold_best, axis=1)

    # Export to latex
    result = style.to_latex(
        convert_css=True,
        hrules=True,
        position_float="centering",
        multirow_align="t",
        caption=caption,
        label=label,
    )
    return result

def copy_latex(latex: str):
    latex = latex.replace("\\", "\\\\")
    latex = latex.replace("\n", "\\n")
    return HTML(f"""<button onclick="navigator.clipboard.writeText('""" + latex + """')">COPY</button>""")


In [35]:
df = clean_table.copy(deep=True)
best_idx = df.groupby("strategy").idxmax().loc[("taskInference"), ("final_accuracy_mean")]

# Create new row
idx = ("taskInference", "", "Best", "AE or VAE")
df.loc[idx] = None
row = df.loc[idx]

for key, value in best_idx.items():
#     df.loc[idx] = "hi"
    row.loc["final_accuracy_std",key] = clean_table.loc[value].loc["final_accuracy_std",key]
    row.loc["final_accuracy_mean",key] = clean_table.loc[value].loc["final_accuracy_mean",key]
    row.loc["count",key] = clean_table.loc[value].loc["count",key]

df = df.reset_index()
best_vs_rest = df[~((df["strategy"] == "taskInference") & (df["prune_proportion"] != "Best"))]
best_vs_rest.set_index(["strategy", "prune_proportion", "replay_buffer", "architecture"], inplace=True)
best_vs_rest

/tmp/ipykernel_2079342/2234278386.py:6: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df.loc[idx] = None


count  \
dataset                                                   S-CIFAR10   
strategy      prune_proportion replay_buffer architecture             
nonContinual                                 AE                10.0   
                                             VAE               10.0   
taskOracle    0.5                            AE                10.0   
finetuning                                   AE                10.0   
                                             VAE               10.0   
replay                         100.0         AE                10.0   
                               1000.0        AE                10.0   
                               10000.0       AE                10.0   
SnB                            1000          FF                10.0   
GR                                           VAE               10.0   
BIR                                          VAE               10.0   
taskInference Best                           AE or VAE         10.0   

                                                                               \
dataset                                                   S-CIFAR100 S-CORe50   
strategy      prune_proportion replay_buffer architecture                       
nonContinual                                 AE                 10.0     10.0   
                                             VAE                10.0     10.0   
taskOracle    0.5                            AE                 10.0     10.0   
finetuning                                   AE                 10.0     10.0   
                                             VAE                10.0     10.0   
replay                         100.0         AE                 10.0     10.0   
                               1000.0        AE                 10.0     10.0   
                               10000.0       AE                 10.0     10.0   
SnB                            1000          FF                 10.0     10.0   
GR                                           VAE                10.0     10.0   
BIR                                          VAE                10.0     10.0   
taskInference Best                           AE or VAE          10.0     10.0   

                                                                    \
dataset                                                   S-FMNIST   
strategy      prune_proportion replay_buffer architecture            
nonContinual                                 AE               10.0   
                                             VAE              10.0   
taskOracle    0.5                            AE               10.0   
finetuning                                   AE               10.0   
                                             VAE              10.0   
replay                         100.0         AE               10.0   
                               1000.0        AE               10.0   
                               10000.0       AE               10.0   
SnB                            1000          FF               10.0   
GR                                           VAE              10.0   
BIR                                          VAE              10.0   
taskInference Best                           AE or VAE        10.0   

                                                                       \
dataset                                                   SE-CIFAR100   
strategy      prune_proportion replay_buffer architecture               
nonContinual                                 AE                  10.0   
                                             VAE                 10.0   
taskOracle    0.5                            AE                  10.0   
finetuning                                   AE                  10.0   
                                             VAE                 10.0   
replay                         100.0         AE                  10.0   
                               1000.0        AE                  10.0   

In [36]:
latex_table = create_latex_table(
        best_vs_rest,
        caption="Best CI-PackNet vs Others \\\\ Mean Final Accuracy $\pm$ One Standard Deviation after 10 runs \\\\ Top: Non-Class-IL Methods, Bottom: Class-IL Methods",
        label="tab:best_vs_rest",
        bolding_ignores_rows=[0, 1, 2]
)
copy_latex(latex_table)

/tmp/ipykernel_2079342/2337888344.py:35: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.drop(columns=["strategy", "replay_buffer", "prune_proportion", "architecture"])


In [37]:
latex_table = create_latex_table(
        clean_table.loc[["taskInference"]],
        caption="CI-PackNet with Different Hyper-Parameters \\\\ Mean Final Accuracy $\pm$ One Standard Deviation after 10 runs",
        label="tab:hp_ci_packnet",
)
copy_latex(latex_table)



/tmp/ipykernel_2079342/2337888344.py:35: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.drop(columns=["strategy", "replay_buffer", "prune_proportion", "architecture"])
